In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pylab as pl

from numpy import linalg
#from qiskit_machine_learning.datasets import ad_hoc_data
#from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit.primitives import Sampler
from qiskit_algorithms.state_fidelities import ComputeUncompute

In [7]:
class Perceptron(object):
	# Este es el método del constructor. Define el número máximo de iteraciones T (épocas) que realizará el perceptrón durante el entrenamiento. Por defecto, se establece en 1.
    def __init__(self, T=1):
        self.T = T
	
	#Este método entrena el perceptrón ajustando los pesos w y el sesgo b a partir de los datos de entrenamiento X y las etiquetas de clase y.
    def fit(self, X, y):
	#Obtiene el número de muestras (n_samples) y el número de características (n_features) de los datos de entrenamiento.
        n_samples, n_features = X.shape
	#Inicializa el vector de pesos w como un vector de ceros con la misma dimensión que el número de características.
        self.w = np.zeros(n_features, dtype=np.float64)
	#Inicializa el sesgo b como cero.
        self.b = 0.0
	# Realiza un bucle sobre el número máximo de iteraciones (T).
        for t in range(self.T):
	#Para cada muestra en los datos de entrenamiento:
            for i in range(n_samples):
	#Verifica si la predicción del perceptrón para la muestra actual es diferente de la etiqueta de clase verdadera.
                if self.predict(X[i])[0] != y[i]:
	#Actualiza los pesos w si la predicción es incorrecta multiplicando la etiqueta de clase por la muestra y sumándola a los pesos.
                    self.w += y[i] * X[i]
	# Actualiza el sesgo b sumando la etiqueta de clase.
                    self.b += y[i]
	
	#Este método calcula la proyección de las muestras X en el hiperplano de separación definido por los pesos w y el sesgo b. Devuelve el resultado de la multiplicación de X por w y sumando b.
    def project(self, X):
        return np.dot(X, self.w) + self.b
	
	#Este método predice las etiquetas de clase para las muestras X dadas.
    def predict(self, X):
	#Convierte X en una matriz bidimensional si no lo es.
        X = np.atleast_2d(X)
	#Devuelve el signo de la proyección de las muestras X, que es la predicción del perceptrón (-1 o 1).
        return np.sign(self.project(X))

In [106]:
import numpy as np

class MultilayerPerceptron(object):
    #Toma tres argumentos obligatorios: layers, una lista que especifica el número de neuronas en cada capa; activation, que especifica la función de activación a usar (por defecto, 'relu'); y T, que especifica el número de iteraciones de entrenamiento (por defecto, 1).
    def __init__(self, layers, activation='relu', T=1):
        self.layers = layers         #Guarda la lista de capas
        self.activation = activation #Guarda el tipo de función de activación
        self.T = T                   #Guarda el número de iteraciones de entrenamiento
        self.hidden_layers = []      #Inicializa una lista vacía para contener los pesos de las capas ocultas.

        # Initialize hidden layers
        for i in range(1, len(layers)):
            self.hidden_layers.append(np.random.randn(layers[i-1] + 1, layers[i])) #Agrega a self.hidden_layers matrices de pesos aleatorias para cada capa oculta, incluyendo una fila extra para el sesgo.
    
    #toma un vector x como entrada y devuelve la salida después de aplicar la función de activación especificada.
    def _activate(self, x):
        if self.activation == 'relu':
            return np.maximum(x, 0)
        elif self.activation == 'sigmoid':
            return 1 / (1 + np.exp(-x))
        elif self.activation == 'tanh': #tangente hiperbólica
            return np.tanh(x)
    
    #las funciones que siguen realizan la propagación hacia adelante, el entrenamiento mediante retropropagación y la predicción, respectivamente
    def _forward_pass(self, X):
        activations = [X]
        input_data = X

        # Forward pass through each hidden layer
        for layer in self.hidden_layers:
            # Add bias term to input data
            input_data = np.hstack((input_data, np.ones((input_data.shape[0], 1))))
            # Compute activations of the layer
            output = self._activate(np.dot(input_data, layer))
            print("layer : ", layer)
            activations.append(output)
            # Update input data for the next layer
            input_data = output

        return activations

    def fit(self, X, y):
    # Iterar sobre el número de épocas
        for t in range(self.T):
            # Iterar sobre cada muestra X en los datos de entrada
            for i in range(len(X)):
                # Realizar un pase hacia adelante para obtener las activaciones de cada capa para la muestra actual X[i:i+1].
                activations = self._forward_pass(X[i:i+1])
                output = activations[-1]  # Salida del modelo

                # Calcular el error y la delta para la retropropagación
                error = y[i] - output
                delta = error
                #print(delta)
                #  #itera sobre cada capa oculta (layer) y sus activaciones correspondientes (activation). Se itera en reversa para comenzar desde la capa de salida y retrocede hacia las capas ocultas.
                for layer, activation in zip(reversed(self.hidden_layers), reversed(activations[:-1])):
                    # Se agrega un término de sesgo (unidades de sesgo) a las activaciones de la capa actual
                    activation_with_bias = np.hstack((activation, np.ones((activation.shape[0], 1))))
                    # #calcula el delta para la capa actual utilizando la matriz de pesos de la capa (layer) transpuesta y se multiplica para propagar el error hacia atrás. 
                    delta = np.dot(delta, layer.T) * (activation_with_bias > 0)
                    # Actualizar los pesos de la capa utilizando el producto punto entre las activaciones de la capa anterior y el delta calculado. Esto ajusta los pesos para minimizar el error en la capa actual durante el proceso de retropropagación.
                    layer += np.dot(activation, delta.T)

                # Imprimir las formas de las matrices activations[-2].T y delta para depuración
                print("Shape de activations[-2].T:", activations[-2].T.shape)
                print("Shape de delta:", delta.shape)


    def predict(self, X):
        activations = self._forward_pass(X)
        return activations[-1]


In [107]:
 #función fit anterior
"""
    def fit(self, X, y):
        for t in range(self.T):
            #print("T: ", self.T) T=100
            for i in range(len(X)):                        #itera sobre cada muestra en el conjunto de datos de entrenamiento X.
                activations = self._forward_pass(X[i:i+1]) #realiza un pase hacia adelante (_forward_pass) en el modelo para obtener las activaciones de cada capa para la muestra actual X[i:i+1].
                output = activations[-1]                   #almacena las activaciones de la capa de salida, que representan las predicciones del modelo para la muestra actual.
                
                # Backpropagation
                error = y[i] - output                      #Calcula el error entre la etiqueta verdadera y[i] y la salida predicha output.
                delta = error                              #Inicializa el delta, que es la cantidad que se utilizará para actualizar los pesos durante el paso de retropropagación
                for layer, activation in zip(reversed(self.hidden_layers), reversed(activations[:-1])): #itera sobre cada capa oculta (layer) y sus activaciones correspondientes (activation). Se itera en reversa para comenzar desde la capa de salida y retrocede hacia las capas ocultas.
                    activation = np.hstack((activation, np.ones((activation.shape[0], 1)))) # Se agrega un término de sesgo (unidades de sesgo) a las activaciones de la capa actual, lo que nos permite modelar mejor las funciones no lineales
                    delta = np.dot(delta, layer.T) * (activation > 0) #calcula el delta para la capa actual utilizando la matriz de pesos de la capa (layer) transpuesta y se multiplica para propagar el error hacia atrás. 

                    print("Shape de activations[-2].T:", activations[-2].T.shape)
                    print("Shape de delta:", delta.shape)
                     #Actualiza los pesos de la capa actual utilizando el producto punto entre las activaciones de la capa anterior y el delta calculado. Esto ajusta los pesos para minimizar el error en la capa actual durante el proceso de retropropagación.
                        #activación capa anterior transpuesta
                    #n12 = np.squeeze(np.asarray(activations[-2].T))
                        #cantidad de error que se retropropaga
                    #n13 = np.squeeze(np.asarray(delta))
                    layer += np.dot(activations[-2].T, delta)
"""

'\n   def fit(self, X, y):\n       for t in range(self.T):\n           #print("T: ", self.T) T=100\n           for i in range(len(X)):                        #itera sobre cada muestra en el conjunto de datos de entrenamiento X.\n               activations = self._forward_pass(X[i:i+1]) #realiza un pase hacia adelante (_forward_pass) en el modelo para obtener las activaciones de cada capa para la muestra actual X[i:i+1].\n               output = activations[-1]                   #almacena las activaciones de la capa de salida, que representan las predicciones del modelo para la muestra actual.\n               \n               # Backpropagation\n               error = y[i] - output                      #Calcula el error entre la etiqueta verdadera y[i] y la salida predicha output.\n               delta = error                              #Inicializa el delta, que es la cantidad que se utilizará para actualizar los pesos durante el paso de retropropagación\n               for layer, acti

In [108]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Paso 1: Cargar y preparar los datos
data = load_iris()
X = data.data
y = data.target

# Escalado de características
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# División en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Paso 2: Crear una instancia del MLP
mlp = MultilayerPerceptron(layers=[4, 5, 3], activation='relu', T=100)

# Paso 3: Entrenar el MLP
mlp.fit(X_train, y_train)

# Paso 4: Evaluar el rendimiento
predictions = mlp.predict(X_test)

# Imprime la precisión del MLP
accuracy = (predictions == y_test).mean()
print("Precisión del MLP:", accuracy)


layer :  [[-0.64099987 -1.06375071  0.33769469  0.77439983  0.87960593]
 [ 0.3133037   2.15749291  1.7577223   0.4117472  -0.66353118]
 [-0.61032138  1.36858309  1.4541339   0.76560783 -0.44034834]
 [-0.47538245 -1.14380683 -0.37958764  0.59168973  0.62755257]
 [ 0.16692007 -0.61051128 -0.25370815 -1.19808516  0.02056355]]
layer :  [[ 1.79979069  1.35948263  0.5534546 ]
 [-0.65721374  0.10114862  2.48782436]
 [-0.06732012  0.72600741  0.56255048]
 [ 0.71045091 -0.45691951  1.14849707]
 [-1.62002644  0.92300954  1.1214564 ]
 [-0.41418795 -0.76203143  2.03538068]]
Traceback (most recent call last):
  Cell In[108], line 21
    mlp.fit(X_train, y_train)
  Cell In[106], line 63 in fit
    layer += np.dot(activation, delta.T)
ValueError: shapes (1,5) and (6,1) not aligned: 5 (dim 1) != 6 (dim 0)

Use %tb to get the full traceback.
